# Hybrid Recommendation System
a) Collaborative Filtering Model

b) Content-based Model

In [1]:
import pandas as pd
from random import randint
import numpy as np
from numpy.linalg import norm 
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from numpy import dot

In [2]:
'''
FORMULA
X(NORMALIZED) = (X - Xminimum)/(Xmaximum - Xminimum)

 min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    
    return [x/max_val for x in data]
'''
#normalizing the input between 0s and 1s
def normalize(data):    
    return (data - np.min(data)/ (np.max(data)- np.min(data)))


In [3]:
''' 
CREATING FUNCTIONS FOR CONTENT BASED RECOMMENDATION MODELS

''' 

#performing one hot encoding
def encoding(df, column):
    new = pd.get_dummies(df[column])
    new.reset_index(drop = True, inplace = True)
    return pd.concat([df, new], axis = 1)

class CBRecommend():
    def __init__(self, df):
        self.df = df
        
    def cosine_sim(self, v1,v2):
        '''
        This function will calculate the cosine similarity between two vectors
        '''
        return sum(dot(v1,v2)/(norm(v1)*norm(v2)))
    
    def recommend(self, book_id, n_rec):
        """
        df (dataframe): The dataframe
        song_id (string): Representing the song name
        n_rec (int): amount of rec user wants
        """
        
        # calculate similarity of input book_id vector w.r.t all other vectors
        inputVec = self.df.loc[book_id].values
        self.df['sim']= self.df.apply(lambda x: self.cosine_sim(inputVec, x.values), axis=1)

        # returns top n user specified books
        return self.df.nlargest(columns='sim',n=n_rec)

In [4]:
'''
CREATING FRUNCTION FOR COLLABORATIVE FILTER MODELLING
'''

def svd(mat, df, factors):
    if not 1<= factors < min(mat.shape):
        raise ValueError("Must be 1 <= factors < min(mat.shape)")
    
    #matrix factorization
    u,s,v = svds(mat, k = factors)
    s = np.diag(s)

    #calculating the prediction ratings
    pred = np.dot(np.dot(u,s),v)
    pred = normalize(pred) #normalizing the predictions

    new_df = pd.DataFrame(pred, columns = df.columns, index = list(df.index) ).transpose()

    return new_df


In [18]:
def generate_data(n_books = 3000, n_genres = 10, n_authors = 450, n_publishers = 50, n_readers = 30000, dataset_size = 100000):
    
    d = pd.DataFrame(
        {
            'book_id' : [randint(1, n_books) for _ in range(dataset_size)],
            'author_id' : [randint(1, n_authors) for _ in range(dataset_size)],
            'book_genre' : [randint(1, n_genres) for _ in range(dataset_size)],
            'reader_id' : [randint(1, n_readers) for _ in range(dataset_size)],
            'num_pages' : [randint(75, 700) for _ in range(dataset_size)],
            'book_rating' : [randint(1, 10) for _ in range(dataset_size)],
            'publisher_id' : [randint(1, n_publishers) for _ in range(dataset_size)],
            'publish_year' : [randint(2000, 2021) for _ in range(dataset_size)],
            'book_price' : [randint(1, 200) for _ in range(dataset_size)],
            'text_lang' : [randint(1,7) for _ in range(dataset_size)]
        }
    ).drop_duplicates()
    return d

In [19]:
df = generate_data(dataset_size = 100000)
df.to_csv('data.csv', index = False)
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
0,1995,232,1,25760,261,3,19,2017,128,6
1,2079,372,2,28057,676,7,16,2009,24,4
2,1561,99,4,7785,655,8,42,2007,8,3
3,2190,256,5,27902,659,1,10,2005,55,7
4,2929,331,9,7196,519,7,46,2013,91,2


In [20]:
df = df.sort_values(by=['book_id'], ascending = True)
df.head()

,book_id,author_id,book_genre,reader_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang
35074,1,118,5,9666,471,4,29,2019,41,7
43993,1,16,1,8395,123,4,43,2020,195,6
79052,1,168,7,28817,135,9,34,2007,142,4
66038,1,74,3,27151,98,10,49,2017,90,3
50530,1,324,9,29332,414,7,3,2000,56,7


In [21]:
if __name__ == '__main__':
    
    df = df

    # normalizing
    df['num_pages_norm'] = normalize(df['num_pages'].values)
    df['book_rating_norm'] = normalize(df['book_rating'].values)
    df['book_price_norm'] = normalize(df['book_price'].values)
    
    # One hot encoding
    df = encoding(df = df, column = 'publish_year')
    df = encoding(df = df, column = 'book_genre')
    df = encoding(df = df, column = 'text_lang')


    # generate a pivot table
    pivot = df.pivot_table(
        columns = 'book_id',
        index = 'reader_id',
        values = 'book_rating'
    ).fillna(0)

    # convert to a csr matrix
    mat = pivot.values
    mat = csr_matrix(mat)
    
    #applying SVD model
    pred_df = svd(mat, pivot, 10)

    ''' 
    pred_df[pred_df.select_dtypes(include=['number']).columns] *= 10
    
    pred_df = pred_df.astype(int)
    print(pred_df)  

    ''' 


    # drop redundant columns
    cols = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang']
    df.drop(columns = cols, inplace = True)
    df.set_index('book_id', inplace = True)

    
    # ran on a sample as an example
    t = df.copy()
    cbr = CBRecommend(df = t)
    print(cbr.recommend(book_id = t.index[0], n_rec = 5))


         author_id  reader_id  publisher_id  num_pages_norm  book_rating_norm  \
book_id                                                                         
19             319      24812            44          676.88          9.888889   
2780           234      17769            30          496.88          7.888889   
665            283      22082            49          615.88          8.888889   
145            267      21054            23          589.88          6.888889   
657            293      23426            45          642.88          5.888889   

         book_price_norm  2000  2001  2002  2003  ...  9  10  1  2  3  4  5  \
book_id                                           ...                         
19            144.994975     0     0     0     1  ...  0   0  0  0  0  0  1   
2780           94.994975     0     0     0     0  ...  0   0  0  0  0  0  0   
665           137.994975     0     0     0     0  ...  0   0  0  0  0  0  0   
145           108.994975     0     0 